#### AMI with the new dataset
The pipeline is run with all the columns of the dataset, we are only keeping 10 features selected with the SelectkBest method in order to do the PCA with fewer number of features.

##### First study

In [ ]:
## Importing the librairies and the already written functions that we will use later on
%run "2 - General\List functions.ipynb"   ## List of general functions that we will need such as visualization, training...

In [ ]:
## Specifying the date to calculate the time taken for the whole process
today = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M')

In [ ]:
data = pd.read_excel(r'C:/Users/33753/OneDrive - Numa Health/Documents - 8 - Medical & Data Science/6 - Théo/01 - Data/6 - Myocarde/df_myocarde_230420_reduced.xlsx',
                     )
data.set_index('Unnamed: 0', inplace= True)

# Removing all the last columns corresponding to the outcomes
data_1 = data.loc[:,data.columns[:data.shape[1]-8]]
outcome = 'MACE 2 yr (death, stroke, MI, hositalizacija(stento restenoz, new stenoz, SN))'
data_1[outcome] = data.loc[:,outcome]

In [ ]:
bio = {'RBC (I) e6': 'RBC',
 'WBC e3 (I)skyriuje' : 'WBC',
 'PMN (%) I' : 'PMN',
 'LYMPH (%)I' : 'LYMPH',
 'Mono (%) I' : 'MONO',
 'EOS (%) I' : 'EOS',
 'BASO (%) I' : 'BASO',
 'PLT.(I)' : 'PLT',
 'Hb mg/dL (I) ' : 'HG',
 'HCT (I)' : 'HCT',
 'MCH (I)' : 'MHC',
 'MCHC (I)' : 'MCHC',
 'K (I)' : 'K',
 'Ca (I)' : 'CA'}

data_1.rename(columns = bio, inplace = True)

data_1.drop(columns = ['Unnamed: 0.1','Cortisol INDEX','Mono/Lym','G/T','Adaptation'], inplace=True)

In [ ]:
data_1 = unit_check(data_1)

In [ ]:
calculation_raw_index(data_1, list_index = [] , Basophile = 'BASO', Eosinophile = 'EOS', Hemoglobine = 'HG', Leucocyte = 'WBC',
                    Lymphocyte = 'LYMPH', Monocyte = 'MONO', Neutrophile = 'PMN', Platelet = 'PLT', TCMHemog = 'MHC',
                    calcium ='CA', potassium = 'K',hematies='RBC')

In [ ]:
feature_biomarkers_3 = ['WBC','HG', 'RBC', 'MHC', 'PMN', 'EOS', 'BASO', 'LYMPH', 'MONO', 'PLT']
feature_index_3 = ['hematies','adapt_ratio', 'locus_ceru_aggr', 'IML', 'IMP', 'genit_thyro', 'loc_ceru_adaptation',
                 'immed_adapt_index', 'cortisol_func', 'cortisol_struc', 'adr_cort_func', 'adr_cort_struc', 'allergy',
                 'evok_hist', 'adapt_perm','immed_adapt_score', 'cata_ana', 'starter', 'alpha_symp_index',
                 'consum_aggr_index', 'locus_ceru_index', 'androgenic', 'AC_OC', 'genital','adaptogene', 'acc_cont', 'pro_inflam']
feature_other_3 = list(filter(lambda x: x not in feature_biomarkers_3, list(data_1.columns)))
feature_other_3 = list(filter(lambda x: x not in feature_index_3, feature_other_3))

In [ ]:
## New columns to be removed based on Kamyar choice 
## 27/04/2023
cols_to_remove = ['STEMI/ non STEMI.1' ,'BMI (kg/m2)' ,'Dyslipidemia' ,'Hypertesion' ,'Smoking' ,
        'DM' ,'CABG' ,'History of IHD' ,'History of heart failue' ,
        'previous PCI' ,'Previous MI' ,'Risk AGG' ,'EF post PCI CAT; <=45 = 1' ,
        'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD' ,'Number of diseased vessel' ,'percentage of culpret vessel; >=50 significant if other physiologic variables are present' ,
        'Coronary dominancy (the artery that feeds the posterior descending artery): 40% RCA, 60% left Circumflex)' ,
        'Cardiac rythm during incharge' ,'in-hospital II - III* AVB' ,'SV during hosp. ' ,'Time until death (1yr)/ Laikas iki mirties (1m)' ,
        'Died in 1 yr / Mire per 1m.' ,'Time until death (2yr)/ Laikas iki mirties (2m)' ,'Na (I)' ,'eGFR 1 (GFG)' ,'Cr 1 (mmol/l)' ,'Cortisol INDEX' ,
        'Death in hospital period; 0=No, 1=Yes' ,'1yr MACE( death. MI. stroke, hospitalisation)' ,'MACE 03.28']

data_1['Mono/Lym']=data_1['MONO']/data_1['LYMPH']
data_1['Thyroid relaunching corrected'] = data_1['Mono/Lym']*data_1['genit_thyro']

feature_biomarkers_4 = ['WBC','HG', 'RBC', 'MHC', 'PMN', 'EOS', 'BASO', 'LYMPH', 'MONO', 'PLT',
                     'eGFR 1 CAT 0 = >90', '?? Troponin [Trop MAX]','All SERUM Cortisol Kortizolis-Serum','CRP']
feature_index_4 = ['IML', 'IMP', 'cortisol_func', 'cortisol_struc', 'consum_aggr_index', 'pro_inflam',
                'adapt_ratio', 'Mono/Lym', 'genit_thyro', 'Thyroid relaunching corrected']
feature_other_4 = list(filter(lambda x: x not in feature_biomarkers_4, list(data_1.columns)))
feature_other_4 = list(filter(lambda x: x not in feature_index_4, feature_other_4))

In [ ]:
## MACE 2yr outcome

feature_sets = {'feature_biomarkers':feature_biomarkers_4,
                'feature_index':feature_index_4, 
                'feature_other':feature_other_4}
today = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M')
for name, feature_set in feature_sets.items():
    df = data_1.loc[~data_1[outcome].isna()].copy()
    for col in cols_to_remove:
        if col in df.columns:
            df.drop(columns = col, inplace=True)
    if 'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD' in df.columns:
        df.drop(columns = ['Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD'], inplace=True)
    if outcome in feature_set:
        feature_set.remove(outcome)

    path = r'C:/Users/33753/OneDrive - Numa Health/Documents - 8 - Medical & Data Science/6 - Théo/03 - Research/5 - Pathologies/6-Myocarde/'
    path += today+'/'+'MACE 2yr'+'/'+name+'/'
    pipeline_classification(df, feature_set, col_to_pred=outcome, cutoff=0, thresh=None, save=True, outdir=path, show=False,
                                scoring='accuracy', loss_function=None, dict_models=None, dict_param_models=None, normalization_method=None,
                                imputation_method='iterative', feature_selection_method='Voting', n_features=4, calcul='No')

In [ ]:
final_features = ['PMN','EOS','LYMPH','immed_adapt_score','cortisol_struc','Thyroid relaunching corrected','genit_thyro','IML','IMP','adapt_ratio'
                '6 month Stroke','Killip klasė (CHF)','heart rate and conduction disarrangement during hospitalization']
df = data_1.loc[~data_1[outcome].isna()].copy()
if 'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD' in df.columns:
    df.drop(columns = ['Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD'], inplace=True)

path = r'C:/Users/33753/OneDrive - Numa Health/Documents - 8 - Medical & Data Science/6 - Théo/03 - Research/5 - Pathologies/6-Myocarde/'
path += today+'/'+'MACE 2yr'+'/'+'final_features'+'/'
pipeline_classification(df, final_features, col_to_pred=outcome, cutoff=0, thresh=None, save=True, outdir=path, show=False,
                            scoring='accuracy', loss_function=None, dict_models=None, dict_param_models=None, normalization_method=None,
                            imputation_method='iterative', feature_selection_method='Voting', n_features=6, calcul='No')

In [ ]:
df = data_1.loc[~data_1[outcome].isna()].copy()
df = fix_decimal_separator(df)
df = convert_to_float(df)
if 'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD' in df.columns:
    df.drop(columns = ['Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD'], inplace=True)
    
a = df.copy()
a.fillna(0, inplace = True)
X = a.drop(outcome, axis = 1)
y = a.loc[:,outcome]

##### AMI with double model and PCA

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
## Importing the librairies and the already written functions that we will use later on
%run "C:\Users\33753\Documents\Machine-Learning\1-Functions\List_Functions.ipynb" ## List of general functions that we will need such as visualization, training...
# %run "2 - General\List functions.ipynb"   ## List of general functions that we will need such as visualization, training...
today = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M')

IndexError: list index out of range

IndexError: list index out of range

In [ ]:
# Specifying the path to save all the plot, datasets etc
path = r'C:/Users/33753/OneDrive - Numa Health/Documents - 8 - Medical & Data Science/6 - Théo/03 - Research/05 - Pathologies/06-Myocarde/'
path += today+'/'+'Test PCA'+'/'

In [ ]:
# Loading the Covid dataset                  
data = pd.read_excel(r'C:/Users/33753/OneDrive - Numa Health/Documents - 8 - Medical & Data Science/6 - Théo/01 - Data/6 - Myocarde/df_myocarde_230420_reduced.xlsx',
                     )
data.set_index('Unnamed: 0', inplace= True)

In [ ]:
# Specifying the name of all the useful features and the outcome
AMI_features = ['RBC (I) e6', 'WBC e3 (I)skyriuje', 'PMN (%) I',
       'LYMPH (%)I', 'Mono (%) I', 'EOS (%) I', 'Cortisol INDEX', 'Mono/Lym',
       'G/T', 'Adaptation', 'BASO (%) I', 'PLT.(I)', 'Hb mg/dL (I) ',
       'MCH (I)', 'Na (I)', '?? Troponin [Trop MAX]',
       'All SERUM Cortisol Kortizolis-Serum', 'FT3 (I)', 'First TnI',
       'Total Cholesterol', 'eGFR 1 (GFG)', 'eGFR 1 CAT 0 = >90',
       'Cr 1 (mmol/l)', 'CRB', 'Hospitalisation Period (days)', 'Age',
       'Age CAT <65 = 0;', 'Sex: 0=MALE, 1=FEMALE', 'STEMI/ non STEMI',
       'STEMI/ non STEMI.1', 'Killip klasė (CHF)', 'BMI (kg/m2)',
       'BMI CAT >30=1', 'Dyslipidemia', 'Hypertesion', 'Smoking', 'DM',
       'Familial IHD', 'CABG', 'History of IHD', 'History of heart failue',
       'previous PCI', 'Previous MI', 'Risk AGG',
       'EF after 24 hr from PCI >=50 NL, <=45 is increased risk of CHF, Mortality',
       'EF post PCI CAT; <=45 = 1',
       'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD',
       'Number of diseased vessel',
       'percentage of culpret vessel; >=50 significant if other physiologic variables are present',
       'Coronary dominancy (the artery that feeds the posterior descending artery): 40% RCA, 60% left Circumflex)',
       'heart rate and conduction disarrangement during hospitalization',
       'Cardiac rythm during incharge', 'in hospital AFib', 'SV during hosp. ',
       'CRP', '6 month Stroke']
outcome = 'MACE 2 yr (death, stroke, MI, hositalizacija(stento restenoz, new stenoz, SN))'

In [ ]:
# A dictionary to rename all the biomarkers to a standard naming
bio = {'RBC (I) e6': 'RBC',
 'WBC e3 (I)skyriuje' : 'WBC',
 'PMN (%) I' : 'PMN',
 'LYMPH (%)I' : 'LYMPH',
 'Mono (%) I' : 'MONO',
 'EOS (%) I' : 'EOS',
 'BASO (%) I' : 'BASO',
 'PLT.(I)' : 'PLT',
 'Hb mg/dL (I) ' : 'HB',
 'HCT (I)' : 'HCT',
 'MCH (I)' : 'MHC',
 'MCHC (I)' : 'MCHC',
 'K (I)' : 'K',
 'Ca (I)' : 'CA'}

data.rename(columns = bio, inplace = True)

In [ ]:
# List of the features to be removed according to the insight of Kamyar Hedayat
toBeRemovedKamyar = ['STEMI/ non STEMI.1' ,'BMI (kg/m2)' ,'Dyslipidemia' ,'Hypertesion' ,'Smoking' ,
        'DM' ,'CABG' ,'History of IHD' ,'History of heart failue' ,
        'previous PCI' ,'Previous MI' ,'Risk AGG' ,'EF post PCI CAT; <=45 = 1' ,
        'Culprit vessel: 0=RCA, 1=Rt Circumflex, 2=LAD' ,'Number of diseased vessel' ,'percentage of culpret vessel; >=50 significant if other physiologic variables are present' ,
        'Coronary dominancy (the artery that feeds the posterior descending artery): 40% RCA, 60% left Circumflex)' ,
        'Cardiac rythm during incharge','SV during hosp. ' ,'Time until death (1yr)/ Laikas iki mirties (1m)' ,
        'Died in 1 yr / Mire per 1m.' ,'Time until death (2yr)/ Laikas iki mirties (2m)' ,'Na (I)' ,'eGFR 1 (GFG)' ,'Cr 1 (mmol/l)' ,'Cortisol INDEX' ,
        'Death in hospital period; 0=No, 1=Yes' ,'1yr MACE( death. MI. stroke, hospitalisation)' ,'MACE 03.28']
data.drop(columns=toBeRemovedKamyar, inplace=True)

# List of features to removed to avoid duplicates
data.drop(columns = ['Unnamed: 0.1','Mono/Lym','G/T','Adaptation'], inplace=True)

In [ ]:
## Preprocessing of the data
df, new_feature_set = engineering(data, AMI_features, col_to_pred=outcome, cutoff=0, normalization_method=None, imputation_method='iterative',
                                    feature_selection_method = None, n_features=5, calcul='Yes')

In [ ]:
# Creating X : the dataset without the expected outcome
X = df.copy()
X.drop(columns=[outcome], inplace=True)
X_index = X.index

# PCA
# Apply PCA to reduce the dimensionality of the data to n_features
pca = PCA()
pca.fit(X)
X = pca.transform(X)

In [ ]:
print("pca.explained_variance_ : \n", pca.explained_variance_, '\n')
print("pca.explained_variance_ratio_ :\n", pca.explained_variance_ratio_, '\n')
print("pca.explained_variance_ratio_.cumsum() :\n", pca.explained_variance_ratio_.cumsum(), '\n')
print("pca.noise_variance_ :\n", pca.noise_variance_, '\n')
print("pca.singular_values_ :\n", pca.singular_values_, '\n')

In [ ]:
# Plot of the explained variance ratio cumulated with lines to better view the 90% threshold
plt.barh(y = np.arange(1, len(list(pca.explained_variance_ratio_.cumsum()))+1), width = list(pca.explained_variance_ratio_.cumsum()), )
plt.axhline(1, color = 'red')
plt.axvline(0.90, color = 'red')

In [ ]:
# Number of facotrial axes to keep
nbFactAxe = len([x for x in list(pca.explained_variance_ratio_.cumsum()) if x < 0.90]) + 1
if nbFactAxe < 5:
    nbFactAxe = 5

In [ ]:
# Getting the most important features from the PCA and saving them
mostImportantFeatures = pd.DataFrame()
i=1
index = range(len(pca.components_[0]))
for comp in range(nbFactAxe):
    s = sorted(index, reverse=True, key=lambda i: abs(pca.components_[comp])[i])[:nbFactAxe]
    S = sorted(abs(pca.components_[comp]), reverse=True)[:nbFactAxe]
    # print(S)
    # print(s[:6])
    mostImportantFeatures[i] = list(df.columns[s])
    i+=1
save_dataframe(outdir = path, dataframe=mostImportantFeatures, file_type='xlsx', name = 'Most important features')

In [ ]:
# Getting the most recurrent features
value_counts = mostImportantFeatures.values.flatten().tolist()
count = pd.Series(value_counts).value_counts()
mostReccurentFeatures = pd.DataFrame(count)
save_dataframe(outdir = path, dataframe=mostReccurentFeatures, file_type='xlsx', name = 'Occurences')

The first new variable is explaining more than 90% of the variance of the data which means that the first variable have more than 90% of the information. 

In [ ]:
# Let's only keep the 11 first variables
fact_ax = list(np.arange(0,nbFactAxe))
dfReduced = pd.DataFrame(X).loc[:,fact_ax]
dfReduced['index']=X_index
dfReduced = dfReduced.set_index('index')
dfReduced[outcome] = df.loc[:,outcome]

In [ ]:
## Visualization des données après le preprocessing
vizualisation(dfReduced, fact_ax, outcome, save=True, outdir=path+'After_Preprocessing/', show=True)

In [ ]:
# Training
X, y = dfReduced.loc[:,list(dfReduced.columns[:-1])], dfReduced.loc[:,outcome]

# Fitting the Decision Tree
dt_model = DecisionTreeClassifier()
dt_model = dt_model.fit(X,y)

# Fitting the Fandom Forest
rf_model = RandomForestClassifier()
rf_model = rf_model.fit(X,y)

# Fitting the XGBoost
xgb_model = XGBClassifier(eval_metric='mlogloss')
xgb_model = xgb_model.fit(X,y)

In [ ]:
# Dictionary to store the results
dict_result = {'DecisionTree':{},
               'RandomForest':{},
               'XGBoost':{}}

In [ ]:
# Declaring the variable that will store the model trained on the survival population
dt_model_survival = DecisionTreeClassifier()
rf_model_survival = RandomForestClassifier()
xgb_model_survival = XGBClassifier(eval_metric='mlogloss')

In [ ]:
# Initialize the StratifiedKFold with 10 splits
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize an empty list to store the residual errors
residual_errors_dt = []
residual_errors_rf = []
residual_errors_xgb = []

# Empty dataframe to store the wrong predictions
dict_error_analysis = {'DecisionTree':[],
               'RandomForest':[],
               'XGBoost':[]}

iter = 1

# Iterate over each fold
for train_index, test_index in skfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Showing the distribution of death and survival in each iteration
    display_distribution_pie(df.iloc[train_index], outcome, save = True,
                             outdir=path+'iter_'+str(iter)+'/', show=False)
    display_distribution_pie(df.iloc[test_index], outcome, save = True,
                             outdir=path+'iter_'+str(iter)+'/', show=False)

    # Initialize and fit the decision tree model
    dt_model_CV = DecisionTreeClassifier()
    rf_model_CV = RandomForestClassifier()
    xgb_model_CV = XGBClassifier(eval_metric='mlogloss')
    
    dt_model_CV = dt_model_CV.fit(X_train, y_train)
    rf_model_CV = rf_model_CV.fit(X_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        xgb_model_CV = xgb_model_CV.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred_dt = dt_model_CV.predict(X_test)
    y_pred_rf = rf_model_CV.predict(X_test)
    y_pred_xgb = xgb_model_CV.predict(X_test)


    # Training a second model on the survival population to determine if a hidden pattern exists
    X_train_survival = X_test.copy()
    X_train_survival['Outcome'] = y_test

    X_train_survival['PredictedOutcomeDt'] = y_pred_dt
    X_train_survival['PredictedOutcomeRf'] = y_pred_rf
    X_train_survival['PredictedOutcomeXgb'] = y_pred_xgb

    X_train_survival_red = X_train_survival.loc[X_train_survival['Outcome']==0]
    y_test_survival_dt = X_train_survival_red['PredictedOutcomeDt']
    y_test_survival_rf = X_train_survival_red['PredictedOutcomeRf']
    y_test_survival_xgb = X_train_survival_red['PredictedOutcomeXgb']

    dt_model_survival = dt_model_survival.fit(X_train_survival_red.loc[:,fact_ax], y_test_survival_dt)
    rf_model_survival = rf_model_survival.fit(X_train_survival_red.loc[:,fact_ax], y_test_survival_rf)
    xgb_model_survival = xgb_model_survival.fit(X_train_survival_red.loc[:,fact_ax], y_test_survival_xgb)


    # Adding the index of the wrong predictions to the corresponding dataframe
    dict_error_analysis['DecisionTree'] += list(X_train_survival[X_train_survival['Outcome'] != X_train_survival['PredictedOutcomeDt']].index)
    dict_error_analysis['RandomForest'] += list(X_train_survival[X_train_survival['Outcome'] != X_train_survival['PredictedOutcomeRf']].index)
    dict_error_analysis['XGBoost'] += list(X_train_survival[X_train_survival['Outcome'] != X_train_survival['PredictedOutcomeXgb']].index)

    # Calculate the residual error (e.g., mean squared error)
    residual_error_dt = mean_squared_error(y_test, y_pred_dt)
    residual_errors_dt.append(round(residual_error_dt, 4))

    residual_error_rf = mean_squared_error(y_test, y_pred_rf)
    residual_errors_rf.append(round(residual_error_rf,4))

    residual_error_xgb = mean_squared_error(y_test, y_pred_xgb)
    residual_errors_xgb.append(round(residual_error_xgb,4))

    # Plotting the confusion matrix for each model
    metrics_dt = display_confusion_matrix(X_test, y_test, dt_model_CV, name=None, y_pred=y_pred_dt, return_metrics=True, save = True,
                             outdir=path+'iter_'+str(iter)+'/', show=False)
    metrics_rf = display_confusion_matrix(X_test, y_test, rf_model_CV, name=None, y_pred=y_pred_rf, return_metrics=True, save = True,
                             outdir=path+'iter_'+str(iter)+'/', show=False)
    metrics_xgb = display_confusion_matrix(X_test, y_test, y_pred_xgb, name=None, y_pred=y_pred_xgb, return_metrics=True, save = True,
                             outdir=path+'iter_'+str(iter)+'/', show=False)
    
    # Saving the results in the dictionary
    dict_result['DecisionTree'][str(iter)] = {'MSE' : round(residual_error_dt,4),
                                    'SB' : round(metrics_dt[1],4),
                                    'ST' : round(metrics_dt[2],4)}
    dict_result['RandomForest'][str(iter)] = {'MSE' : round(residual_error_rf,4),
                                    'SB' : round(metrics_rf[1],4),
                                    'ST' : round(metrics_rf[2],4)}
    dict_result['XGBoost'][str(iter)] = {'MSE' : round(residual_error_xgb,4),
                                    'SB' : round(metrics_xgb[1],4),
                                    'ST' : round(metrics_xgb[2],4)}

    iter+=1

# Print the residual error for the current fold and the average residual error across all folds
print("Residual error (fold):", residual_errors_dt)
print("Average residual error:", round(sum(residual_errors_dt) / len(residual_errors_dt),4))

# Print the residual error for the current fold and the average residual error across all folds
print("Residual error (fold):", residual_errors_rf)
print("Average residual error:", round(sum(residual_errors_rf) / len(residual_errors_rf),4))

# Print the residual error for the current fold and the average residual error across all folds
print("Residual error (fold):", residual_errors_xgb)
print("Average residual error:", round(sum(residual_errors_xgb) / len(residual_errors_xgb),4))

In [ ]:
# Getting the rows that were falsely predicted
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['DecisionTree'],:], file_type = 'xlsx', name='Error_Analysis_CV_dt')
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['RandomForest'],:], file_type = 'xlsx', name='Error_Analysis_CV_rf')
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['XGBoost'],:], file_type = 'xlsx', name='Error_Analysis_CV_xgb')

In [ ]:
# Calculate the mean value and the STD of all the metrics
df_AVG = pd.DataFrame()
for typeModel in ['DecisionTree','RandomForest','XGBoost']:
    MSE_list = []
    SB_list=[]
    ST_list=[]
    for value in dict_result[typeModel].values():
        MSE_list.append(value['MSE'])
        SB_list.append(value['SB'])
        ST_list.append(value['ST'])
    df_AVG.loc['AVG',typeModel] = str({'MSE':round(np.mean(MSE_list),4),
                                'SB':round(np.mean(SB_list),4),
                                'ST':round(np.mean(ST_list),4)})
    df_AVG.loc['STD',typeModel] = str({'MSE':round(np.std(MSE_list),4),
                                'SB':round(np.std(SB_list),4),
                                'ST':round(np.std(ST_list),4)})

In [ ]:
# Saving the results as a dataframe
df_result = pd.DataFrame(dict_result)
df_result = pd.concat([df_result,df_AVG])

In [ ]:
# Initialize the LeaveOneOUt cross validation
skfold = LeaveOneOut()

# Initialize an empty list to store the residual errors
residual_errors_dt = []
residual_errors_rf = []
residual_errors_xgb = []

dict_PredictedOutcome = {'DecisionTree':[],
                        'RandomForest':[],
                        'XGBoost':[]}

pred_test = pd.DataFrame()
dict_error_analysis = {'DecisionTree':[],
               'RandomForest':[],
               'XGBoost':[]}

# Iterate over each fold
for train_index, test_index in skfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize and fit the decision tree model
    dt_model_CV = DecisionTreeClassifier()
    rf_model_CV = RandomForestClassifier()
    xgb_model_CV = XGBClassifier(eval_metric='mlogloss')
    
    dt_model_CV = dt_model_CV.fit(X_train, y_train)
    rf_model_CV = rf_model_CV.fit(X_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        xgb_model_CV = xgb_model_CV.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred_dt = dt_model_CV.predict(X_test)
    dict_PredictedOutcome['DecisionTree'].append(y_pred_dt)
    y_pred_rf = rf_model_CV.predict(X_test)
    dict_PredictedOutcome['RandomForest'].append(y_pred_rf)
    y_pred_xgb = xgb_model_CV.predict(X_test)
    dict_PredictedOutcome['XGBoost'].append(y_pred_xgb)
    pred_test = pd.concat([pred_test,y_test])

    # Calculate the residual error (e.g., mean squared error)
    residual_error_dt = mean_squared_error(y_test, y_pred_dt)
    residual_errors_dt.append(residual_error_dt)

    residual_error_rf = mean_squared_error(y_test, y_pred_rf)
    residual_errors_rf.append(residual_error_rf)

    residual_error_xgb = mean_squared_error(y_test, y_pred_xgb)
    residual_errors_xgb.append(residual_error_xgb)

pred_test.loc[:,'PredictedOutcomeDt'] = [int(arr[0]) for arr in dict_PredictedOutcome['DecisionTree']]
pred_test.loc[:,'PredictedOutcomeRf'] = [int(arr[0]) for arr in dict_PredictedOutcome['RandomForest']]
pred_test.loc[:,'PredictedOutcomeXgb'] = [int(arr[0]) for arr in dict_PredictedOutcome['XGBoost']]

# Adding the index of the wrong predictions to the corresponding dataframe
dict_error_analysis['DecisionTree'] += list(pred_test[pred_test[0] != pred_test['PredictedOutcomeDt']].index)
dict_error_analysis['RandomForest'] += list(pred_test[pred_test[0] != pred_test['PredictedOutcomeRf']].index)
dict_error_analysis['XGBoost'] += list(pred_test[pred_test[0] != pred_test['PredictedOutcomeXgb']].index)

In [ ]:
# Getting the rows that were falsely predicted
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['DecisionTree'],:], file_type = 'xlsx', name='Error_Analysis_LOO_dt')
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['RandomForest'],:], file_type = 'xlsx', name='Error_Analysis_LOO_rf')
save_dataframe(outdir=path+'Error analysis/', dataframe=data.loc[dict_error_analysis['XGBoost'],:], file_type = 'xlsx', name='Error_Analysis_LOO_xgb')

In [ ]:
df_LOO = pd.DataFrame()
for typeModel in ['DecisionTree','RandomForest','XGBoost']:

    # Calculate the confusion matrix for all folds
    cm = confusion_matrix(pred_test[0], dict_PredictedOutcome[typeModel])

    # Calculate sensitivity (recall)
    sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # Calculate specificity
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

    # Calculate mean squared error
    mse = mean_squared_error(pred_test[0], dict_PredictedOutcome[typeModel])

    df_LOO.loc['LOO',typeModel] = str({'MSE':round(mse,4),
                                     'SB':round(sensitivity,4),
                                     'ST':round(specificity,4)})

    print("Sensitivity:", round(sensitivity,4))
    print("Specificity:", round(specificity,4))
    print("Mean Squared Error:", round(mse,4))

In [ ]:
# Saving the dataframe with all the results
df_result = pd.concat([df_result,df_LOO])
df_result = df_result.applymap(lambda x: str(x).replace('{', '').replace('}', ''))
save_dataframe(outdir=path, dataframe=df_result, file_type = 'xlsx', name='results')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

# Step 1: Perform PCA on the dataset
# data = pd.read_csv('your_dataset.csv')  # Load your dataset
# X = data.drop('class', axis=1)  # Extract the feature columns
# y = data['class']  # Extract the class column

# pca = PCA(n_components=X.shape[1])
# X_pca = pca.fit_transform(X)

# Step 2: Extract the first 3 principal components
principal_components = X.loc[:,:3]
principal_components.reset_index(inplace=True)

# Step 3: Create a scatter plot to visualize the individuals
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Color code the data points based on the class
colors = {0.0: 'red', 1.0: 'blue'}
for i, target in enumerate(np.unique(y)):
    indices = np.where(y == target)
    ax.scatter(principal_components.loc[indices, 0],
               principal_components.loc[indices, 1],
               principal_components.loc[indices, 2],
               c=colors[target], label=target)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.legend()
plt.show()

In [ ]:
path

In [ ]:
df_stat = data.describe()
save_dataframe(outdir=path, dataframe=df_stat, file_type = 'xlsx', name='Description des données')

END of PCA study